# Singleton

**Singleton** to wzorzec kreacyjny zakładający istnienie co najwyżej *N* (najczęściej 1) instancji danej klasy, która musi być wszędzie dostępna. Singleton'u należy używać tam, gdzie utworzenie drugiego obiektu tej klasy może mieć negatywne skutki. Zastosowanie wzorca singleton powinno być dokładnie uzasadnione, gdyż jego nadużywanie może stanowić **antywzorzec**.

Przypadki zastosowania:
- logger,
- dziennik zdarzeń,
- bufor,
- obiekt ustawień,
- obiekt łączący z bazą danych lub zewnętrzną usługą,
- szyna danych.

## Implementacja

### Wersja podstawowa

Implementacja wersji podstawowej singleton'u zostanie zbudowana na założeniu weryfikacji czy inna instancja obiektu nie została już utworzona. W przypadku istnienia, zostanie zwrócona utworzona instancja. Wersja ta zakłada istnienie maksymalnie 1 instancji klasy.

In [9]:
from typing import Self

In [10]:
class Singleton:
    _instance: Self = None
    
    def __new__(cls, *args: list, **kwargs: dict) -> Self:
        if cls._instance is None:
            instance = super().__new__(cls, *args, **kwargs)
            cls._instance = instance
            
        return cls._instance

In [11]:
x = Singleton()
y = Singleton()

In [12]:
x is y

True

### Wersja rozszerzona

Wersja rozszerzona implementacji singleton'u zakłada istnienie maksymalnie *N* instancji klasy. W przypadku utworzenia większej liczby instancji, będą zwracane pozycje według schematu: `instance_no % N`.

In [ ]:
class SingletonExtended:
    _instances: list = []
    _instances_max: int = 3
    _instances_actual: int = -1
    
    def __new__(cls, *args: list, **kwargs: dict) -> Self:
        if len(cls._instances) < cls._instances_max:
            instance = super().__new__(cls, *args, **kwargs)
            cls._instances.append(instance)
        
        cls._instances_actual += 1
        cls._instances_actual = cls._instances_actual % cls._instances_max

        return cls._instances[cls._instances_actual]

In [17]:
for i in range(7):
    print(f"{i}: {id(SingletonExtended())}")

0: 4408469760
1: 4406010064
2: 4406015504
3: 4408469760
4: 4406010064
5: 4406015504
6: 4408469760


### Zastosowanie w innych klasach

Klasa reprezentująca singleton może być z powodzeniem zastosowana jako metaklasa dla docelowej klasy, której liczebność instancji powinna być ograniczona.

In [29]:
class SingletonMeta(type):
    _instance: Self = None
    
    def __call__(cls, *args: list, **kwargs: dict) -> Self:
        if cls._instance is None:
            instance = super().__call__(*args, **kwargs)
            cls._instance = instance
            
        return cls._instance

In [30]:
class A(metaclass=SingletonMeta):
    def foo(self) -> str:
        return "foo"

In [31]:
a = A()
a.foo()

'foo'

In [32]:
for _ in range(5):
    print(id(A()))

4408471440
4408471440
4408471440
4408471440
4408471440


## Podsumowanie

Wzorzec singleton jest wykorzystywany wszędzie tam, gdzie istnieje konieczność kontrolowania liczby instancji klasy. Takie podejście rodzi pewne wady i zalety.

Wady:
- komplikacje przy współbieżności,
- klasa kontroluje instancje oprócz rzeczy które ma na celu,
- trudny w testowaniu.

Zalety:
- szybka implementacja,
- brak zmian po stronie projektu,
- stan singletonów jest globalny,
- leniwe tworzenie instancji: obiekt nie zostanie utworzony, jeżeli nigdy nie zostanie użyty.